### Deep CNN on the CIFAR10 datasets (due: July 3)

In [1]:
import keras, os
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 100
data_augumentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [3]:
# (X, Y training data), (X, Y testing data)
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [4]:
# (train(test)_samples, ...)
print('(X, Y)_train_shape:', X_train.shape, '|', Y_train.shape)
print('(X, Y)_test_shape :', X_test.shape, '|', Y_test.shape)

(X, Y)_train_shape: (50000, 32, 32, 3) | (50000, 1)
(X, Y)_test_shape : (10000, 32, 32, 3) | (10000, 1)


In [5]:
# Convert class vectors to binary class matrixes
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [6]:
model = Sequential()

# inner layer (if "layers" is 0, then this is the input layer)
for layers in range(2):
    model.add(Conv2D(32, (3, 3), 
                     padding='same', 
                     input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

# output layer
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
# initiate RMSprop optimizer
optimizer = keras.optimizers.rmsprop(lr=0.0001, decay=0.00001)

# train the model with RMSprop
model.compile(loss='categorical_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [9]:
if not data_augumentation:
    print('=== Not using data_augmentation ===')
    model.fit(X_train, Y_train,
             batch_size=batch_size,
             epochs=epochs,
             validation_data=(X_test, Y_test),
             shuffle=True)
else:
    print('=== Real-time data_augmentation ===')
    data_genarator = ImageDataGenerator(featurewise_center=False,
                                       samplewise_center=False,
                                       featurewise_std_normalization=False,
                                       samplewise_std_normalization=False,
                                       zca_epsilon=0.00001,
                                       zca_whitening=False,
                                       rotation_range=0,
                                       width_shift_range=0.1,
                                       height_shift_range=0.1,
                                       shear_range=0.,
                                       zoom_range=0.,
                                       channel_shift_range=0.,
                                       fill_mode='nearest',
                                       cval=0.,
                                       horizontal_flip=True,
                                       vertical_flip=False,
                                       rescale=None,
                                       preprocessing_function=None,
                                       data_format=None,
                                       validation_split=0.0)
    
    data_genarator.fit(X_train)
    generator = data_genarator.flow(X_train, Y_train,
                                    batch_size = batch_size)
    model.fit_generator(generator,
                        steps_per_epoch=len(generator),
                        epochs=epochs,
                        validation_data=(X_test, Y_test),
                        workers=4)

=== Real-time data_augmentation ===
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1563/1563 [==============================] - 194s 124ms/step - loss: 1.9659 - acc: 0.2722 - val_loss: 1.6664 - val_acc: 0.4084
Epoch 2/100
1563/1563 [==============================] - 198s 126ms/step - loss: 1.6786 - acc: 0.3861 - val_loss: 1.5125 - val_acc: 0.4510
Epoch 3/100
1563/1563 [==============================] - 187s 120ms/step - loss: 1.5597 - acc: 0.4266 - val_loss: 1.4228 - val_acc: 0.4908
Epoch 4/100
1563/1563 [==============================] - 181s 116ms/step - loss: 1.4746 - acc: 0.4662 - val_loss: 1.3112 - val_acc: 0.5223
Epoch 5/100
1563/1563 [==============================] - 183s 117ms/step - loss: 1.4173 - acc: 0.4881 - val_loss: 1.2760 - val_acc: 0.5399
Epoch 6/100
1563/1563 [==============================] - 183s 117ms/step - loss: 1.3698 - acc: 0.5053 - val_loss: 1.2810 - val_acc: 0.5400
Epoch 7/100
1563/1563 [==============================] - 199s 127ms/step - loss: 1

KeyboardInterrupt: 

In [ ]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at \"%s\".' % model_path)

In [ ]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])